# Process

## 參數的邊界
將每一層的 weight, bias, scale 印出來 

In [1]:
import os
from prettytable import PrettyTable

def read_values_from_file(file_path):
    with open(file_path, 'r') as file:
        values = [float(line.strip()) for line in file]
    return values

def find_min_max_values(folder_path, output_file):
    table = PrettyTable()
    table.field_names = ["File", "Type", "Minimum Value", "Maximum Value", "Max Weight * Scale"]
    table.align["File"] = "l"
    table.align["Type"] = "l"
    table.align["Minimum Value"] = "r" 
    table.align["Maximum Value"] = "r"
    table.align["Max Weight * Scale"] = "r"

    base_path = os.path.join(folder_path, '')

    for root, dirs, files in os.walk(folder_path):
        max_weight_scale = float('-inf')
        for file in files:
            if file in ['quantized_weights.txt', 'quantized_bias.txt', 'quantization_scale.txt', 'parameters.txt']:
                file_path = os.path.join(root, file)
                values = read_values_from_file(file_path)
                min_value = min(values)
                max_value = max(values)
                relative_path = file_path.replace(base_path, '').replace('__packed_params__packed_params', '_pk').replace('_projection', '')
                
                if file == 'quantized_weights.txt':
                    value_type = 'Weight'
                    scale_file = os.path.join(root, 'quantization_scale.txt')
                    if os.path.exists(scale_file):
                        scale = read_values_from_file(scale_file)[0]
                        max_weight_scale = max(max_weight_scale, max_value * scale)
                elif file == 'quantized_bias.txt':
                    value_type = 'Bias'
                elif file == 'quantization_scale.txt':
                    value_type = 'Scale'
                else:
                    value_type = 'Parameter'
                
                display_path = relative_path
                for x in ["encoder_0_params\\", "embedding_params", "classifier_params"]:
                    display_path = display_path.replace(x, '')
                
                table.add_row([display_path, value_type, min_value, max_value, f'{max_weight_scale:.6f}' if value_type == 'Weight' else '-'])

    print(table)

    with open(output_file, 'w') as out_file:
        out_file.write(str(table))

# Specify the path to the folder containing the quantized model
folder_path = '32float_quantized'

# Specify the output file path
output_file = 'min_max_values.txt'

# Call the function to find the minimum and maximum values and write to file
find_min_max_values(folder_path, output_file)


+---------------------------------------------------------------+-----------+---------------+---------------+--------------------+
| File                                                          | Type      | Minimum Value | Maximum Value | Max Weight * Scale |
+---------------------------------------------------------------+-----------+---------------+---------------+--------------------+
| \classifier_1_pk\quantization_scale.txt                       | Scale     |    0.00233354 |    0.00233354 |                  - |
| \classifier_1_pk\quantized_bias.txt                           | Bias      |    0.05804596 |     0.1120506 |                  - |
| \classifier_1_pk\quantized_weights.txt                        | Weight    |        -128.0 |         112.0 |           0.261356 |
| \embedding_0_pk\quantization_scale.txt                        | Scale     |    0.00993039 |    0.00993039 |                  - |
| \embedding_0_pk\quantized_bias.txt                            | Bias      |   -0.

## Convert to binary
觀察上表可以發現，Bias, scale 的值落在 -1~1 之間，所以用~~完全小數表示~~，要使用

**`Linear`**
- Scale: 16-bit, 1-bit integer part + 15-bit fraction part
- Bias: 16-bit, 1-bit integer part + 15-bit fraction part
- Weight: 8-bit, 2-bit integer, 6-bit fraction

**`cls_token`, `positional_encoding`**

16-bit, 4-bit integer part, 12-bit fraction part

In [2]:
import os

def float_to_fixed16(float_value, integer_bits, fraction_bits):
    # Convert float to 16-bit fixed-point representation with specified integer and fraction bits
    fixed16_value = int(round(float_value * (2**fraction_bits)))
    if fixed16_value < 0:
        fixed16_value = (abs(fixed16_value) ^ 0xFFFF) + 1
    binary = format(fixed16_value & 0xFFFF, '016b')
    return binary

def process_file(file_path, output_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        processed_lines = []
        for line in lines:
            try:
                value = float(line.strip())
                if file_path.endswith('_bias.txt'):
                    # Convert bias values to 16-bit fixed-point with 1-bit integer and 15-bit fraction
                    fixed16_binary = float_to_fixed16(value, 1, 15)
                    processed_lines.append(fixed16_binary + '\n')
                elif file_path.endswith('quantization_scale.txt'):
                    # Convert quantization_scale values to 16-bit fixed-point with 1-bit integer and 15-bit fraction
                    fixed16_binary = float_to_fixed16(value, 1, 15)
                    processed_lines.append(fixed16_binary + '\n')
                elif 'embedding_cls_token' in file_path or 'positional_encoding' in file_path:
                    # Convert embedding_cls_token and positional_encoding values to 16-bit fixed-point with 4-bit integer and 12-bit fraction
                    fixed16_binary = float_to_fixed16(value, 4, 12)
                    processed_lines.append(fixed16_binary + '\n')
                elif value.is_integer():
                    # Convert integer to two's complement representation
                    integer = int(value)
                    if integer < 0:
                        integer = (~abs(integer) + 1) & 0xFF
                    processed_lines.append(format(integer, '08b') + '\n')
                else:
                    # Convert float to 8-bit fixed-point representation with 1-bit integer and 7-bit fraction
                    fixed8_value = int(round(value * (2**7)))
                    if fixed8_value < 0:
                        fixed8_value = (~abs(fixed8_value) + 1) & 0xFF
                    processed_lines.append(format(fixed8_value, '08b') + '\n')
            except ValueError:
                processed_lines.append(line)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as file:
        file.writelines(processed_lines)

def process_directory(input_directory, output_directory):
    for root, dirs, files in os.walk(input_directory):
        for file in files:
            if file.endswith('.txt'):
                input_file_path = os.path.join(root, file)
                output_file_path = os.path.join(output_directory, os.path.relpath(input_file_path, input_directory))
                process_file(input_file_path, output_file_path)

# Specify the path to the input directory (32float_quantized)
input_directory = '32float_quantized'
# Specify the path to the output directory (8bit_fixed_point)
output_directory = '8bit_fixed_point'

# Process all .txt files in the input directory and its subdirectories
process_directory(input_directory, output_directory)